In [ ]:
%%capture

!pip install category_encoders==2.*

In [ ]:
import pandas as pd
import numpy as np
from category_encoders import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier, plot_tree

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving monsters.csv to monsters.csv


In [ ]:
df = pd.read_csv('monsters.csv')
print(df.shape)
df.head(10)

(1000, 10)


,Unnamed: 0,Name,Type,Level,Rarity,Damage,Health,Energy,Sanity,Timestamp
0,0,Hook Horror,Demonic,8,Rank 1,8d4+4,30.41,32.20,32.01,2023-05-04 09:04:55
1,1,Magma Elemental,Elemental,7,Rank 2,7d6+1,39.31,40.85,40.40,2023-05-04 09:04:55
2,2,Sapphire Drake,Dragon,6,Rank 3,6d8+3,49.73,51.58,49.64,2023-05-04 09:04:55
3,3,Emerald Archfey,Fey,19,Rank 1,19d4,75.55,77.66,76.74,2023-05-04 09:04:55
4,4,Kobold Knight,Devilkin,4,Rank 1,4d4+1,16.38,16.66,14.93,2023-05-04 09:04:55
5,5,Silver Drake,Dragon,4,Rank 2,4d6,23.93,24.07,23.35,2023-05-04 09:04:55
6,6,Sapphire Demon,Demonic,1,Rank 0,1d2,1.84,2.02,1.74,2023-05-04 09:04:55
7,7,Lightning Elemental,Elemental,17,Rank 0,17d2+3,33.79,34.91,34.07,2023-05-04 09:04:55
8,8,Mummy Lord,Undead,5,Rank 0,5d2+1,9.66,10.97,10.60,2023-05-04 09:04:55
9,9,Brass Faerie,Fey,3,Rank 1,3d4+1,12.84,12.64,13.21,2023-05-04 09:04:55


In [ ]:
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
df.head()

,Name,Type,Level,Rarity,Damage,Health,Energy,Sanity,Timestamp
0,Hook Horror,Demonic,8,Rank 1,8d4+4,30.41,32.20,32.01,2023-05-04 09:04:55
1,Magma Elemental,Elemental,7,Rank 2,7d6+1,39.31,40.85,40.40,2023-05-04 09:04:55
2,Sapphire Drake,Dragon,6,Rank 3,6d8+3,49.73,51.58,49.64,2023-05-04 09:04:55
3,Emerald Archfey,Fey,19,Rank 1,19d4,75.55,77.66,76.74,2023-05-04 09:04:55
4,Kobold Knight,Devilkin,4,Rank 1,4d4+1,16.38,16.66,14.93,2023-05-04 09:04:55


In [ ]:
df = df.drop(['Name', 'Timestamp', 'Damage'], axis=1)
df.head()

,Type,Level,Rarity,Health,Energy,Sanity
0,Demonic,8,Rank 1,30.41,32.20,32.01
1,Elemental,7,Rank 2,39.31,40.85,40.40
2,Dragon,6,Rank 3,49.73,51.58,49.64
3,Fey,19,Rank 1,75.55,77.66,76.74
4,Devilkin,4,Rank 1,16.38,16.66,14.93


In [ ]:
target = 'Rarity'
y = df[target]
x = df.drop(columns=target)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(x, y , test_size=0.2, random_state =42)

# Random Forest Model

In [ ]:
model_rf = make_pipeline(
    OrdinalEncoder(),
    RandomForestClassifier(random_state=42, n_jobs=-1,max_samples=0.5)
)

model_rf.fit(X_train, y_train)

Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['Type'],
                                mapping=[{'col': 'Type',
                                          'data_type': dtype('O'),
                                          'mapping': Elemental    1
Dragon       2
Fey          3
Devilkin     4
Demonic      5
Undead       6
NaN         -2
dtype: int64}])),
                ('randomforestclassifier',
                 RandomForestClassifier(max_samples=0.5, n_jobs=-1,
                                        random_state=42))])

In [ ]:
training_acc = model_rf.score(X_train, y_train)
val_acc = model_rf.score(X_val, y_val)

print('Training Accuracy Score:', training_acc)
print('Validation Accuracy Score:', val_acc)

Training Accuracy Score: 0.9975
Validation Accuracy Score: 0.94


# Decision Tree Classifier

In [ ]:
dt_model = make_pipeline(
    OrdinalEncoder(),
    DecisionTreeClassifier(random_state=42, max_depth=16)
)

dt_model.fit(X_train, y_train)

Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['Type'],
                                mapping=[{'col': 'Type',
                                          'data_type': dtype('O'),
                                          'mapping': Elemental    1
Dragon       2
Fey          3
Devilkin     4
Demonic      5
Undead       6
NaN         -2
dtype: int64}])),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(max_depth=16, random_state=42))])

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
baseline_acc = y_train.value_counts(normalize=True).max()
print('Baseline Accuracy Score:', baseline_acc)

training_acc = accuracy_score(y_train, dt_model.predict(X_train))
val_acc = accuracy_score(y_val, dt_model.predict(X_val))

print('Training Accuracy Score:', training_acc)
print('Validation Accuracy Score:', val_acc)

Baseline Accuracy Score: 0.28875
Training Accuracy Score: 1.0
Validation Accuracy Score: 0.915


# Logistic Regression

In [ ]:
from category_encoders import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [ ]:
lr_model = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    StandardScaler(),
    LogisticRegression(max_iter=100)
)

lr_model.fit(X_train, y_train)

Pipeline(steps=[('onehotencoder',
                 OneHotEncoder(cols=['Type'], use_cat_names=True)),
                ('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [ ]:
print('model_lr accuracy score for training', accuracy_score(y_train, lr_model.predict(X_train)))
print('model_lr accuracy score for val', accuracy_score(y_val, lr_model.predict(X_val))) 

model_lr accuracy score for training 0.89875
model_lr accuracy score for val 0.905


After using a logistic regression, decision tree and random forest model, based on the accuracy scores for training and validation data, I will be moving forward with the random forest model. The random forest model will consist of many decision trees which is another reason I would like to use this model. The average of many decision trees is taken in to build the random forest model, and an increase in the number of trees used can increase the precision. The decision tree model I used above seems to be overfit to our data. 